In [14]:
!pip install -U scikit-learn
!pip install verta
!pip install Minio
!pip install pandas

Requirement already up-to-date: scikit-learn in /opt/app-root/lib/python3.6/site-packages (0.23.2)
You should consider upgrading via the '/opt/app-root/bin/python3.6 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/app-root/bin/python3.6 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/app-root/bin/python3.6 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/app-root/bin/python3.6 -m pip install --upgrade pip' command.


In [68]:
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import joblib
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datetime import datetime

from minio import Minio
from minio.error import ResponseError
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

from verta import Client
# Common Functions - Will be supplied via package
def get_s3_server():
    minioClient = Minio('minio-ml-workshop:9000',
                    access_key='minio',
                    secret_key='minio123',
                    secure=False)

    return minioClient

def record_metrics(classifier, expereiment_name, accuracy_score, hyperparameters):
    client = Client("http://chart-1603715522-webapp:3000")
    proj = client.set_project("HDFC DEmo PRoject")
    client.set_experiment("Iris Classifier")
    run = client.set_experiment_run(expereiment_name)
    
    for key, value in hyperparameters.items():
        run.log_hyperparameters({key : value})
    run.log_metric('accuracy', accuracy_score)
    run.log_tags([classifier])

def visualise_iris_data(iris):
    ''' 0 == setosa 
        1 == versicolor
        2 == viginica '''
    #     print(iris)
    
    df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
    print(df.describe())
    df["target"] = iris.target
    print(df.head(500))

In [69]:
# Get Experiment ID
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d%Y%H%M%S%f")
experiment_id = 'IrisScikit' + timestampStr


In [70]:
def train_and_save_model():
#     model = LogisticRegression()
    model = RandomForestClassifier(n_estimators=10)
    
    print('Training model...')
    model.fit(X1, y1)
    print('Model trained!')
    
    accuracy = model.score(X2, y2)
    print("Accuracy: %.2f%%" % (accuracy*100.0))   
#     record_metrics('LogisticREgression', experiment_id, accuracy, {'regularisation': 0.5})
    record_metrics('RandomForrests', experiment_id, accuracy, {'n_estimators': 10})                                   
    

    filename_p = 'IrisClassifier.sav'
    print('Saving model in %s' % filename_p)
    joblib.dump(model, filename_p)
    print('Model saved!')

In [71]:
# if __name__ == "__main__":
print('Loading iris data set...')
iris = datasets.load_iris()
X, y = iris.data, iris.target
X1, X2, y1, y2 = train_test_split(X, y, random_state=0, train_size=0.6)
print('Dataset loaded!')
visualise_iris_data(iris)
#train_and_save_model()

Loading iris data set...
Dataset loaded!
       sepal length (cm)  sepal width (cm)  petal length (cm)  \
count         150.000000        150.000000         150.000000   
mean            5.843333          3.057333           3.758000   
std             0.828066          0.435866           1.765298   
min             4.300000          2.000000           1.000000   
25%             5.100000          2.800000           1.600000   
50%             5.800000          3.000000           4.350000   
75%             6.400000          3.300000           5.100000   
max             7.900000          4.400000           6.900000   

       petal width (cm)  
count        150.000000  
mean           1.199333  
std            0.762238  
min            0.100000  
25%            0.300000  
50%            1.300000  
75%            1.800000  
max            2.500000  
     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.

In [6]:
minioClient = get_s3_server()
minioClient.fput_object(bucket_name='models', object_name=experiment_id  +'/IrisClassifier.sav' , file_path='./IrisClassifier.sav')

('2401ccb188dc043ebf98ab59c43c3d1f', None)